In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sp
from scipy.integrate import odeint

In [41]:
#Initializations

x0 = np.array([10,0,np.pi/2,-60,0,-np.pi/2])
u = np.array([2, -np.pi/18, 12, np.pi/25])

# x0 = np.array([
#             [10],
#             [0],
#             [np.pi/2],
#             [-60],
#             [0],
#             [-np.pi/2]
#             ])

# u = np.array([
#             [2],
#             [-np.pi/18],
#             [12],
#             [np.pi/25]
#             ])


L = 0.5
num_time_steps = 400
delta_t = 0.1
time = np.linspace(0, num_time_steps*delta_t, num_time_steps)


In [42]:
#Generate Nominal trajectory with no noise

def model(x, t, u, L):
    e_g = x[0]
    n_g = x[1]
    theta_g = x[2]
    e_a = x[3]
    n_a = x[4]
    theta_a = x[5]
    v_g = u[0]
    phi_g = u[1]
    v_a = u[2]
    w_a = u[3] 
    dxdt = [0]*6
    dxdt[0] = v_g*np.cos(theta_g)
    dxdt[1] = v_g*np.sin(theta_g)
    dxdt[2] = (v_g/L)*np.tan(phi_g)
    dxdt[3] = v_a*np.cos(theta_a)
    dxdt[4] = v_a*np.sin(theta_a)
    dxdt[0] = w_a
    return dxdt


nominal_trajectory = odeint(model, x0, time, args = (u,L))

In [65]:
def get_CT_Jacobians(x,u):
    
    A = np.array([
            [0, 0, -u[0]*np.sin(x[2]), 0, 0, 0],
            [0, 0, u[0]*np.cos(x[2]), 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, -u[2]*np.sin(x[5])],
            [0, 0, 0, 0, 0, u[2]*np.cos(x[5])],
            [0, 0, 0, 0, 0, 0]])

    
    B = np.array([
            [np.cos(x[2]), 0, 0, 0],
            [np.sin(x[2]), 0, 0, 0],
            [np.tan(u[1])/L, u[0]/(L*(np.cos(u[1])**2)), 0, 0],
            [0, 0, np.cos(x[5]), 0],
            [0, 0, np.sin(x[5]), 0],
            [0, 0, 0, 1]
            ])    
    
    Ga = np.eye(6)
    
    denominator_1 = 1 + ((x[4]-x[1])/(x[3]-x[0]))**2
    numerator_11 = (x[4]-x[1]) / ((x[3] - x[0])**2)
    C11 = numerator_11/denominator_1
    
    numerator_12 = -1/(x[3]-x[0])
    C12 = numerator_12/denominator_1
    
    C14 = -1*C11
    C15 = -1*C12
    
    denominator_2 = ( (x[0]-x[3])**2 + (x[1]-x[4])**2 ) ** 0.5
    C21 = (x[0]-x[3])/denominator_2
    C22 = (x[1]-x[4])/denominator_2
    C24 = -1*C21
    C25 = -1*C22
    
    C = np.array([
            [C11, C12, -1, C14, C15, 0],
            [C21, C22, 0, C24, C25, 0],
            [C11, C12, 0, C14, C15, -1],
            [0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0]
            ])

    D = 0
    
    return A,B, Ga, C, D

def get_DT_Jacobians(x,u,dt):
    
    A,B,Gamma,C,D = get_CT_Jacobians(x,u)
    
    F = np.eye(6) + dt*A
    G = dt*B
    Omega = delta_t*Gamma
    H = C
    M = D
    
    return F,G,Omega,H,M

#F,G,Omega,H,M = get_DT_Jacobians(x0,u,delta_t)

In [66]:
def get_K_perturbations(initial_perturbation,nominal_trajectory,u,delta_t,K):
    
    F0,G0,Omega0,H0,M0 = get_DT_Jacobians(nominal_trajectory[0],u,delta_t)
    delta_x0 = np.array(initial_perturbation).reshape(len(initial_perturbation),1) 
    delta_x_list = [delta_x0]
    delta_y_list = [np.dot(H0,delta_x0)]
    delta_xk = delta_x0 
    delta_uk = np.zeros(4).reshape(4,1)
    for k in range(1,K):
        xk_nominal = nominal_trajectory[k]
        uk_nominal = u
        Fk,Gk,Omegak,Hk,Mk = get_DT_Jacobians(xk_nominal,uk_nominal,delta_t)
        delta_xk_plus1 = np.dot(Fk,delta_xk) + np.dot(Gk,delta_uk)
        delta_yk = np.dot(Hk,delta_xk)
        delta_xk = delta_xk_plus1
        delta_x_list.append(delta_xk)
        delta_y_list.append(delta_yk)
        
    return delta_x_list,delta_y_list

initial_perturbation = 0.01*np.ones(len(x0))
delta_x_list, delta_y_list = get_K_perturbations(initial_perturbation, nominal_trajectory, u, delta_t, num_time_steps)

In [76]:
non_linear_trajectory_with_perturbed_x0 = odeint(model, x0+initial_perturbation, time, args = (u,L))
non_linear_trajectory_with_perturbed_x0

array([[ 1.00100000e+01,  1.00000000e-02,  1.58079633e+00,
        -5.99900000e+01,  1.00000000e-02, -1.56079633e+00],
       [ 1.00225979e+01,  2.10395057e-01,  1.51008877e+00,
        -5.99779701e+01, -1.19294737e+00, -1.56079633e+00],
       [ 1.00351957e+01,  4.09930394e-01,  1.43938120e+00,
        -5.99659403e+01, -2.39589474e+00, -1.56079633e+00],
       ...,
       [ 1.50113525e+01,  6.38988870e-01, -2.64901055e+01,
        -5.52141397e+01, -4.77560105e+02, -1.56079633e+00],
       [ 1.50239504e+01,  4.41699452e-01, -2.65608130e+01,
        -5.52021099e+01, -4.78763053e+02, -1.56079633e+00],
       [ 1.50365482e+01,  2.42394341e-01, -2.66315206e+01,
        -5.51900800e+01, -4.79966000e+02, -1.56079633e+00]])

In [81]:
linearized_DT_states = []
for k in range(0,len(nominal_trajectory)):
    xk = nominal_trajectory[k] + delta_x_list[k]
    linearized_dt_states.append(xk)
    
linearized_DT_measurements = []

In [82]:
linearized_dt_states

[array([[ 1.00100000e+01,  1.00000000e-02,  1.58079633e+00,
         -5.99900000e+01,  1.00000000e-02, -1.56079633e+00],
        [ 1.00100000e+01,  1.00000000e-02,  1.58079633e+00,
         -5.99900000e+01,  1.00000000e-02, -1.56079633e+00],
        [ 1.00100000e+01,  1.00000000e-02,  1.58079633e+00,
         -5.99900000e+01,  1.00000000e-02, -1.56079633e+00],
        [ 1.00100000e+01,  1.00000000e-02,  1.58079633e+00,
         -5.99900000e+01,  1.00000000e-02, -1.56079633e+00],
        [ 1.00100000e+01,  1.00000000e-02,  1.58079633e+00,
         -5.99900000e+01,  1.00000000e-02, -1.56079633e+00],
        [ 1.00100000e+01,  1.00000000e-02,  1.58079633e+00,
         -5.99900000e+01,  1.00000000e-02, -1.56079633e+00]]),
 array([[ 10.02060286,   0.20833924,   1.50809376, -59.991995  ,
          -1.19500252,  -1.56279133],
        [ 10.02273916,   0.21047554,   1.51023006, -59.9898587 ,
          -1.19286622,  -1.56065503],
        [ 10.02259787,   0.21033425,   1.51008877, -59.99      ,
 

In [72]:
len(x0)

6

In [73]:
nominal_trajectory.shape

(400, 6)

In [74]:
len(delta_x_list)

400

In [80]:
len(nominal_trajectory)

400